In [5]:
from model.srgan import generator
from model import resolve_single
import utils

model = generator()
model.load_weights('weights/srgan/gan_generator.h5')

#lr = utils.load_image('demo/0805x4-crop.png')
#sr = resolve_single(model, lr)
#utils.plot_sample(lr, sr)

In [6]:
print(type(lr)) # INPUT
print(type(sr)) # OUTPUT

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
scale = 30
width = int(frame.shape[1] * (scale/100))
height = int(frame.shape[0] * (scale/100))
dim = (width, height)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    crop = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    upscale = resolve_single(model, crop).numpy()
    
    # Display the resulting frame
    cv2.imshow('input', frame)
    cv2.imshow('cropped', crop)
    cv2.imshow('upscale', upscale)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()